In [6]:
# We will be using these PDF loaders but you can check out other loaded documents
from langchain_community.document_loaders import PyMuPDFLoader

In [7]:
# This loader uses PyMuPDF
loader_py = PyMuPDFLoader("../uploads\\7210843.pdf")

In [8]:
pages_py = loader_py.load()

In [9]:
len(pages_py)

19

In [10]:
print(pages_py)

[Document(metadata={'source': '../uploads\\7210843.pdf', 'file_path': '../uploads\\7210843.pdf', 'page': 0, 'total_pages': 19, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'iTextSharp™ 5.5.13.2 ©2000-2020 iText Group NV (AGPL-version)', 'creationDate': "D:20211231212940+05'30'", 'modDate': "D:20211231212940+05'30'", 'trapped': ''}, page_content='Thyrocare,\n5CA-711, 3rd Floor,\nHRBR 2nd Block,\nHennur, Bengaluru-560043\nPROCESSED AT :\nNAME\n \nSELF\nVIVEK  BARNWAL(32Y/M)\nREF. BY\nTEST ASKED\nAAROGYAM 1.3\n:\n:\n:\nFLAT NO - B-106 TRIFECTA STARLIGHT APARTMENT \nGARUDCHARPALYA MAHADEVPURA 560048\nPATIENTID\n: VB18181374\nHOME COLLECTION :\nTEST NAME\nUNITS\nVALUE\nTECHNOLOGY \nCYSTATIN C\nIMMUNOTURBIDIMETRY\nmg/L\n 0.83\nReference Range :-\n<= 60 years: <= 1.03 mg/L\n> 60 years : < 1.50 mg/L\nClinical significance\nCystatin c, is a small 13-kda protein and is a member of the cysteine proteinase inhibitor family, it is produce

In [1]:
from langchain.schema import Document

In [2]:
import pandas as pd

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
import os
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [5]:
GROQ_API_KEY= os.getenv("GROQ_API_KEY")

In [6]:
import pytesseract
from PIL import Image
from pdf2image import convert_from_path
import os

# Path to the PDF file
pdf_file = "../uploads\\7210843.pdf"

# Convert PDF to images
images = convert_from_path(pdf_file, dpi=300)

content = []
page_num = []


# Iterate through pages
for i, image in enumerate(images):
    # Perform OCR on the image

    text = pytesseract.image_to_string(image)
    #page_num
    page_num.append(i+1)
    # page_content = text   
    content.append(text)

# Directory where images will be saved
output_dir = '../output_directory'

# Ensure the output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir, exist_ok= True)

# Save each page as an image
for i, page in enumerate(images):
    image_path = os.path.join(output_dir, f'page_{i + 1}.png')
    page.save(image_path, 'PNG')
    

In [7]:
data = pd.DataFrame({
    "page": page_num,
    "content": content
})

In [8]:
data.head()

,page,content
0,1,"PROCESSED AT:\n\nThyrocare, Thyro C are\n\n5CA..."
1,2,"PROCESSED AT:\n\nThyrocare, Thyro C are\n\n5CA..."
2,3,"PROCESSED AT:\nThyrocare,\n\n5CA-711, 3rd Floo..."
3,4,"PROCESSED AT:\nThyrocare,\n\n5CA-711, 3rd Floo..."
4,5,"Thyrocare, Thyrocare’\n\n5CA-711, 3rd Floor,\n..."


In [9]:
data_list = []
for index, row in data.iterrows():
        # Construct an Object with 'product_name' and 'review' attributes
        object = {
        'page': row['page'],
        'content': row['content']
        }
        # Append the object to the list
        data_list.append(object)

docs = []

for entry in data_list:
    metadata = {"source": pdf_file, "file_path": pdf_file, 'total_pages': len(page_num), 'format': 'PDF 1.4', 'title': '', "page": entry['page']}
    doc = Document(page_content= entry['content'], metadata= metadata)
    docs.append(doc)    
    


In [10]:
docs

[Document(metadata={'source': '../uploads\\7210843.pdf', 'file_path': '../uploads\\7210843.pdf', 'total_pages': 19, 'format': 'PDF 1.4', 'title': '', 'page': 1}, page_content='PROCESSED AT:\n\nThyrocare, Thyro C are\n\n5CA-711, 3rd Floor,\nHRBR 2nd Block, The Trust. The Truth.\n\nHennur, Bengaluru-560043\n\nCorporate office : Thyrocare Technologies Limited, 9 D-37/3, TTC MIDC, Turbhe, Navi Mumbai - 400 703\n© 022 - 3090 0000 / 67123400 {99870666333 i wellness@thyrocare.com ) www.thyrocare.com\n\nNAME : VIVEK BARNWAL(32Y/M) HOME COLLECTION :\n\nREF. BY » SELF FLAT NO - B-106 TRIFECTA STARLIGHT APARTMENT\nTEST ASKED " AAROGYAM 1.3 GARUDCHARPALYA MAHADEVPURA 560048\nPATIENTID : VB18181374\n\nTEST NAME TECHNOLOGY VALUE UNITS\nCYSTATIN C IMMUNOTURBIDIMETRY 0.83 mg/L\n\nReference Range :-\n\n<= 60 years: <= 1.03 mg/L\n> 60 years : < 1.50 mg/L\n\nClinical significance\n\nCystatin c, is a small 13-kda protein and is a member of the cysteine proteinase inhibitor family, it is produced at a cons

In [11]:
from langchain.vectorstores import Qdrant
from langchain.embeddings import HuggingFaceBgeEmbeddings

d:\General Practitioner\venv\lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [12]:
# Load the embedding model 
model_name = "BAAI/bge-large-en"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

d:\General Practitioner\venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [13]:
url = "http://localhost:6333"
collection_name="vector_db"
qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    url=url,
    prefer_grpc=False,
    collection_name= collection_name
)

In [14]:
retriever= qdrant.as_retriever(search_kwargs={"k": 3})
retriever

VectorStoreRetriever(tags=['Qdrant', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_community.vectorstores.qdrant.Qdrant object at 0x000001F925355300>, search_kwargs={'k': 3})

In [15]:
qdrant.similarity_search("About my Cholestrol levels, do i have to do anything? any exercise or diet?")

[Document(metadata={'source': '../uploads\\7210843.pdf', 'file_path': '../uploads\\7210843.pdf', 'total_pages': 19, 'format': 'PDF 1.4', 'title': '', 'page': 6, '_id': '26915e66-01de-47ce-aaa4-4f9095473728', '_collection_name': 'vector_db'}, page_content='Thyrocare, Thyrocare\n\n5CA-711, 3rd Floor,\nHRBR 2nd Block, The Trust. The Truth.\n\nHennur, Bengaluru-560043\n\nCorporate office : Thyrocare Technologies Limited, 9 D-37/3, TTC MIDC, Turbhe, Navi Mumbai - 400 703\n© 022 - 3090 0000 / 67123400 {99870666333 i wellness@thyrocare.com ) www.thyrocare.com\n\nNAME : VIVEK BARNWAL(32Y/M) HOME COLLECTION :\n\nREF. BY . SELF FLAT NO - B-106 TRIFECTA STARLIGHT APARTMENT\nTEST ASKED  AAROGYAM 1.3 GARUDCHARPALYA MAHADEVPURA 560048\nPATIENTID : VB18181374\n\nTEST NAME TECHNOLOGY VALUE UNITS\nLIPOPROTEIN (A) [LP(A)] IMMUNOTURBIDIMETRY 33.01 mg/dl\n\nReference Range :-\n\nADULTS : < 30.0 MG/DL\n\nINTERPRETATION:\n\nDETERMINATION OF LPA MAY BE USEFUL TO GUIDE MANAGEMENT OF INDIVIDUALS WITH A FAMILY 

In [57]:
GENERAL_PRACTITIONER = """
    You are a General Medical Practitioner who is supposed to answer user queries. 
    You are provided medical reports of patient. Answer queries based on the medical reports.
    Only answer based on the context provided. I am expecting detailed answers and make them more understandable.
    context - {context}
    I you have context then only give answer based on that.
    Only return the helpful answer below and nothing else.
    Helpful answer:
    """


In [58]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="mixtral-8x7b-32768",
    api_key=GROQ_API_KEY
)

In [59]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.chat_history import BaseChatMessageHistory
from langchain.memory import ChatMessageHistory
from langchain.prompts import ChatPromptTemplate

In [60]:
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

In [61]:
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [62]:
history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualize_q_prompt)

In [63]:
chat_history= []

In [64]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", GENERAL_PRACTITIONER),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [65]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [66]:
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [67]:
store = {}
def get_session_history(session_id: str)-> BaseChatMessageHistory:
  if session_id not in store:
    store[session_id]= ChatMessageHistory()
  return store[session_id]

In [68]:
from langchain_core.runnables.history import RunnableWithMessageHistory

In [71]:
chain_with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",)

In [74]:
response= chain_with_message_history.invoke({"input": "About my Cholestrol levels, do i have to do anything? any exercise or diet?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

In [75]:
print(response)

Yes, based on your cholesterol levels, it would be advisable to make some lifestyle modifications. Your total cholesterol is 192 mg/dl, which is above the desirable range. Your LDL or "bad" cholesterol is 131 mg/dl, which is also above the optimal range.

Here are some recommendations:

1. Diet: Follow a heart-healthy diet rich in fruits, vegetables, whole grains, lean proteins, and healthy fats. Limit your intake of saturated and trans fats, cholesterol, and added sugars.
2. Physical Activity: Engage in regular physical activity. Aim for at least 30 minutes of moderate-intensity exercise on most days of the week.
3. Weight Management: If you are overweight, losing even a small amount of weight can help improve your cholesterol levels.
4. Limit Alcohol: Limit your alcohol intake. Excessive alcohol can increase your triglyceride levels and contribute to weight gain.
5. Medication: Depending on your overall risk for heart disease, your doctor may also prescribe medication to help lower y

In [78]:
response= chain_with_message_history.invoke(
    {"input": "Explain me this is points?",},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

In [80]:
print(response)

Sure, I'd be happy to explain those points in more detail:

1. Diet: A heart-healthy diet is rich in fruits, vegetables, whole grains, lean proteins, and healthy fats. It's important to limit your intake of saturated and trans fats, cholesterol, and added sugars. Saturated fats are found in animal products such as meat, butter, and cheese, as well as in some plant-based oils like coconut and palm oil. Trans fats are often found in processed foods, fried foods, and baked goods. Eating too much of these fats can raise your cholesterol levels.

Eating a diet rich in fiber, particularly soluble fiber, can help lower your cholesterol levels. Soluble fiber is found in foods such as oats, barley, fruits, vegetables, and legumes.

2. Physical Activity: Regular physical activity can help raise your HDL or "good" cholesterol levels and lower your LDL or "bad" cholesterol levels. Aim for at least 30 minutes of moderate-intensity exercise on most days of the week. Moderate-intensity exercise inclu

In [43]:
response= chain_with_message_history.invoke(
    {"input": "what was my previous question?",},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]
print(response)

Based on the context provided, there is no information about your previous question. However, I can provide you with a helpful answer about how to manage high homocysteine levels, as it might be a topic of interest based on the information you have previously shared.

High homocysteine levels, or hyperhomocysteinemia, can be managed through various approaches. Here are some recommendations for managing high homocysteine levels:

1. Dietary changes: Consuming a balanced diet rich in fruits, vegetables, whole grains, lean proteins, and low-fat dairy products can help reduce homocysteine levels. Foods high in folate, such as leafy green vegetables, legumes, and fortified cereals, should be emphasized. Additionally, limiting the intake of alcohol, processed foods, and high-fat animal products can also be beneficial.
2. Supplementation: Taking B vitamins, particularly folic acid (vitamin B9), vitamin B6, and vitamin B12, can help lower homocysteine levels. The recommended daily doses are:
	